In [1]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [2]:
load_dotenv(override=True)

True

In [5]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name = "Search Agent",
    tools = [WebSearchTool(search_context_size="low")],
    instructions=INSTRUCTIONS,
    model = "gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required")  #By this be make it mandatory for the agent to run the specified tool
)

In [6]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

As of June 2025, several AI agent frameworks have emerged, each offering unique capabilities for developing intelligent, autonomous systems.

**LangChain** is a widely adopted framework that enables developers to build applications powered by large language models (LLMs). It offers a modular design with extensive integrations, supporting multi-step workflows and complex agent loops. Key features include native support for various LLMs, a rich ecosystem of tools like Google Search and SQL, and capabilities for memory and conversation tracking. However, advanced workflows may require a solid understanding of prompt engineering and LLM behavior. ([medium.com](https://medium.com/%40elisowski/top-ai-agent-frameworks-in-2025-9bcedab2e239?utm_source=openai))

**AutoGen**, developed by Microsoft, focuses on automating the generation of code, models, and processes for complex workflows. It leverages LLMs to assist developers in building, fine-tuning, and deploying AI solutions with minimal manual coding. AutoGen's architecture consists of three layers: Core, AgentChat, and Extensions, facilitating scalable and distributed networks of agents. It also provides tools like AutoGen Bench for performance assessment and AutoGen Studio for no-code agent development. ([blog.mechcloud.io](https://blog.mechcloud.io/top-5-ai-agent-frameworks-in-2025?utm_source=openai))

**CrewAI** is designed for orchestrating role-playing AI agents in collaborative tasks, emphasizing simplicity and minimal setup. It allows for role-based agent assignments, simple implementation without complex dependencies, and collaborative workflows. CrewAI has gained popularity in customer service and marketing automation sectors. ([datacamp.com](https://www.datacamp.com/blog/best-ai-agents?utm_source=openai))

**LangGraph**, an extension of LangChain, introduces a graph-based system for managing data flow between agents. It's particularly useful for applications requiring decision loops or path changes based on new input, supporting branching, looping, and conditional flows. LangGraph is ideal for designing systems that mimic human decision-making in areas like loan processing and workflow automation. ([phyniks.com](https://phyniks.com/blog/top-7-agentic-ai-frameworks-in-2025?utm_source=openai))

**Eliza** is an open-source, Web3-friendly AI agent operating system that integrates seamlessly with blockchain applications. It allows for reading and writing blockchain data, interacting with smart contracts, and is fully controlled by the user through regular TypeScript programs. Eliza aims to make deploying Web3 applications effortless. ([arxiv.org](https://arxiv.org/abs/2501.06781?utm_source=openai))

**AutoAgent** is a fully automated, zero-code framework for LLM agents, enabling users to create and deploy agents through natural language alone. It comprises components like Agentic System Utilities, an LLM-powered Actionable Engine, a Self-Managing File System, and a Self-Play Agent Customization module. AutoAgent serves as a versatile multi-agent system for general AI assistants, demonstrating superior performance in generalist multi-agent tasks. ([arxiv.org](https://arxiv.org/abs/2502.05957?utm_source=openai))

These frameworks reflect the rapid advancements in AI agent development, offering diverse tools and approaches for building intelligent, autonomous systems. 